In [1]:
import RNNmpc.Forecasters.s5_forecaster

In [2]:
model = RNNmpc.Forecasters.s5_forecaster.S5Layer(3, 10)

tensor([-0.5000+31.3133j, -0.5000-31.3133j, -0.5000+9.1299j, -0.5000-9.1299j,
        -0.5000+4.0976j, -0.5000-4.0976j, -0.5000+1.7091j, -0.5000-1.7091j,
        -0.5000+0.3994j, -0.5000-0.3994j], dtype=torch.complex128) tensor([[-0.0976-0.0313j, -0.0976+0.0313j, -0.0803-0.0933j, -0.0803+0.0933j,
         -0.0904+0.1483j, -0.0904-0.1483j,  0.1588-0.2501j,  0.1588+0.2501j,
          0.5970+0.0000j,  0.5970+0.0000j],
        [-0.1650-0.0648j, -0.1650+0.0648j, -0.0998-0.1856j, -0.0998+0.1856j,
         -0.2425+0.1490j, -0.2425-0.1490j,  0.4018+0.0000j,  0.4018+0.0000j,
         -0.2534+0.3428j, -0.2534-0.3428j],
        [-0.2002-0.1099j, -0.2002+0.1099j, -0.0168-0.2653j, -0.0168+0.2653j,
         -0.3140-0.1138j, -0.3140+0.1138j, -0.0469+0.3867j, -0.0469-0.3867j,
          0.0730-0.3295j,  0.0730+0.3295j],
        [-0.2073-0.1721j, -0.2073+0.1721j,  0.1644-0.2564j,  0.1644+0.2564j,
          0.0112-0.3519j,  0.0112+0.3519j, -0.2879-0.2133j, -0.2879+0.2133j,
          0.0155+0.2865j,  0.01

In [5]:
model.lambda_vec.data

tensor([-0.5000+31.3133j, -0.5000-31.3133j, -0.5000+9.1299j, -0.5000-9.1299j,
        -0.5000+4.0976j, -0.5000-4.0976j, -0.5000+1.7091j, -0.5000-1.7091j,
        -0.5000+0.3994j, -0.5000-0.3994j], dtype=torch.complex128)

In [10]:
import torch
A_legsn = torch.zeros((10, 10), dtype=torch.complex128)
for n in range(10):
    for k in range(10):
        if n == k:
            A_legsn[n,k] = -1/2
        if n > k:
            A_legsn[n,k] = -((n+1/2)**(1/2)) * ((k+1/2)**(1/2))
        if n < k:
            A_legsn[n,k] = ((n+1/2)**(1/2)) * ((k+1/2)**(1/2))

lambda_vec, eig_vecs = torch.linalg.eig(A_legsn)

In [11]:
lambda_vec

tensor([-0.5000+31.3133j, -0.5000-31.3133j, -0.5000+9.1299j, -0.5000-9.1299j,
        -0.5000+4.0976j, -0.5000-4.0976j, -0.5000+1.7091j, -0.5000-1.7091j,
        -0.5000+0.3994j, -0.5000-0.3994j], dtype=torch.complex128)

In [3]:
import torch
a = torch.ones((10,10))

In [5]:
a.dtype

torch.float32

In [7]:
a = a.type(torch.complex128)

In [9]:
a.dtype

torch.complex128

In [ ]:
def make_HiPPO(N):
    """ Create a HiPPO-LegS matrix.
        From https://github.com/srush/annotated-s4/blob/main/s4/s4.py
        Args:
            N (int32): state size
        Returns:
            N x N HiPPO LegS matrix
    """
    P = np.sqrt(1 + 2 * np.arange(N))
    A = P[:, np.newaxis] * P[np.newaxis, :]
    A = np.tril(A) - np.diag(np.arange(N))
    return -A

In [ ]:
make_HiPPO(3) - A_legs

array([[ 0.0000000e+00, -0.0000000e+00, -0.0000000e+00],
       [ 0.0000000e+00,  4.4408921e-16, -0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00, -8.8817842e-16]])

In [ ]:
A_legsn = np.zeros((3,3))
for n in range(3):
    for k in range(3):
        if n == k:
            A_legsn[n,k] = -1/2
        if n > k:
            A_legsn[n,k] = -((n+1/2)**(1/2)) * ((k+1/2)**(1/2))
        if n < k:
            A_legsn[n,k] = ((n+1/2)**(1/2)) * ((k+1/2)**(1/2))

In [ ]:
A_legsn

array([[-0.5       ,  0.8660254 ,  1.11803399],
       [-0.8660254 , -0.5       ,  1.93649167],
       [-1.11803399, -1.93649167, -0.5       ]])

In [ ]:
def make_NPLR_HiPPO(N):
    """
    Makes components needed for NPLR representation of HiPPO-LegS
     From https://github.com/srush/annotated-s4/blob/main/s4/s4.py
    Args:
        N (int32): state size

    Returns:
        N x N HiPPO LegS matrix, low-rank factor P, HiPPO input matrix B

    """
    # Make -HiPPO
    hippo = make_HiPPO(N)

    # Add in a rank 1 term. Makes it Normal.
    P = np.sqrt(np.arange(N) + 0.5)

    # HiPPO also specifies the B matrix
    B = np.sqrt(2 * np.arange(N) + 1.0)
    return hippo, P, B
hippo, P, B = make_NPLR_HiPPO(3)

In [ ]:
A_legs

array([[-1.        ,  0.        ,  0.        ],
       [-1.73205081, -2.        ,  0.        ],
       [-2.23606798, -3.87298335, -3.        ]])

In [ ]:
A_legs + P.reshape(-1,1) @ P.reshape(1,-1) - A_legsn

array([[ 1.11022302e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00, -4.44089210e-16,  4.44089210e-16]])

In [ ]:
def make_DPLR_HiPPO(N):
    """
    Makes components needed for DPLR representation of HiPPO-LegS
     From https://github.com/srush/annotated-s4/blob/main/s4/s4.py
    Note, we will only use the diagonal part
    Args:
        N:

    Returns:
        eigenvalues Lambda, low-rank term P, conjugated HiPPO input matrix B,
        eigenvectors V, HiPPO B pre-conjugation

    """
    A, P, B = make_NPLR_HiPPO(N)

    S = A + P[:, np.newaxis] * P[np.newaxis, :]


    S_diag = np.diagonal(S)
    Lambda_real = np.mean(S_diag) * np.ones_like(S_diag)

    # Diagonalize S to V \Lambda V^*
    Lambda_imag, V = np.linalg.eigh(S * -1j)

    P = V.conj().T @ P
    B_orig = B
    B = V.conj().T @ B
    return Lambda_real + 1j * Lambda_imag, P, B, V, B_orig

In [ ]:
b = make_DPLR_HiPPO(3)

[[-0.5         0.8660254   1.11803399]
 [-0.8660254  -0.5         1.93649167]
 [-1.11803399 -1.93649167 -0.5       ]]


In [ ]:
A_legsn

array([[-0.5       ,  0.8660254 ,  1.11803399],
       [-0.8660254 , -0.5       ,  1.93649167],
       [-1.11803399, -1.93649167, -0.5       ]])

In [ ]:
np.linalg.eig(A_legsn)[0]

array([-0.5+0.j        , -0.5+2.39791576j, -0.5-2.39791576j])

In [ ]:
b[0]

array([-0.5-2.39791576e+00j, -0.5-2.23971846e-16j, -0.5+2.39791576e+00j])

In [ ]:
b[0][0] - np.linalg.eig(A_legsn)[0][2]

-8.881784197001252e-16j

In [ ]:
A_legsn = np.zeros((3,3))
for n in range(3):
    for k in range(3):
        if n == k:
            A_legsn[n,k] = -1/2
        if n > k:
            A_legsn[n,k] = -((n+1/2)**(1/2)) * ((k+1/2)**(1/2))
        if n < k:
            A_legsn[n,k] = ((n+1/2)**(1/2)) * ((k+1/2)**(1/2))

In [ ]:
A_legsn

array([[-0.5       ,  0.8660254 ,  1.11803399],
       [-0.8660254 , -0.5       ,  1.93649167],
       [-1.11803399, -1.93649167, -0.5       ]])

In [ ]:
eigs_func, P_func, B_func, V_func, B_orig_func = make_DPLR_HiPPO(3)

[[-0.5         0.8660254   1.11803399]
 [-0.8660254  -0.5         1.93649167]
 [-1.11803399 -1.93649167 -0.5       ]]


In [ ]:
eigs_me, v_me = np.linalg.eig(A_legsn)

In [ ]:
eigs_me

array([-0.5+0.j        , -0.5+2.39791576j, -0.5-2.39791576j])

In [ ]:
v_me

array([[-0.80757285+0.j        , -0.22116293-0.35355339j,
        -0.22116293+0.35355339j],
       [ 0.4662524 +0.j        ,  0.12768848-0.61237244j,
         0.12768848+0.61237244j],
       [-0.36115756+0.j        ,  0.65938047+0.j        ,
         0.65938047-0.j        ]])

In [ ]:
import torch
torch.linalg.eig(torch.tensor(A_legsn))

torch.return_types.linalg_eig(
eigenvalues=tensor([-0.5000+0.0000j, -0.5000+2.3979j, -0.5000-2.3979j],
       dtype=torch.complex128),
eigenvectors=tensor([[-0.8076+0.0000j, -0.2212-0.3536j, -0.2212+0.3536j],
        [ 0.4663+0.0000j,  0.1277-0.6124j,  0.1277+0.6124j],
        [-0.3612+0.0000j,  0.6594+0.0000j,  0.6594-0.0000j]],
       dtype=torch.complex128))

In [ ]:
eigs_func

array([-0.5-2.39791576e+00j, -0.5-2.23971846e-16j, -0.5+2.39791576e+00j])

Keeping eigs me constant, eigs func goes 1, 2, 0.

In [ ]:
np.linalg.norm(v_me @ np.diag(eigs_me) @ v_me.conj().T - A_legsn)

4.718957269898597e-16

In [ ]:
np.linalg.norm(V_func @ np.diag(eigs_func) @ V_func.conj().T - A_legsn)

1.7709714597310653e-15

In [ ]:
V_func

array([[-0.41702883+0.00000000e+00j,  0.80757285+0.00000000e+00j,
         0.41702883+0.00000000e+00j],
       [-0.45144695+4.33012702e-01j, -0.4662524 +1.11022302e-16j,
         0.45144695+4.33012702e-01j],
       [ 0.3496893 +5.59016994e-01j,  0.36115756+1.42011982e-16j,
        -0.3496893 +5.59016994e-01j]])

In [ ]:
v_me

array([[-0.80757285+0.j        , -0.22116293-0.35355339j,
        -0.22116293+0.35355339j],
       [ 0.4662524 +0.j        ,  0.12768848-0.61237244j,
         0.12768848+0.61237244j],
       [-0.36115756+0.j        ,  0.65938047+0.j        ,
         0.65938047-0.j        ]])